For deterministic feature hashing,

- run this script before '===========NEED TO RUN ALL ABOVE' 
- export .py file of this script
- replace '!' with '#!' in .py file
- run .py as below
```
export PYTHONHASHSEED=0
python3 1_sparkprocess_avazuctr_withdash.py
```
- run this script after '==========NEED TO COMMMENT OUT'



In [ ]:
# check memory in megabytes
# !free -m

# sleep for 3 hours
# import time
# time.sleep(3*60*60) # in seconds

## Script options

In [ ]:
first_time = True

full_data = True

copy_full_data = True
copy_sample_data = False

In [ ]:

if first_time:
  from google.colab import drive
  drive.mount("/content/drive")
  pass

Mounted at /content/drive


## Set up Spark

In [ ]:
if first_time:
  !apt-get update
  !apt-get install openjdk-8-jdk-headless -qq > /dev/null
  # !wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
  # !tar xf spark-2.4.7-bin-hadoop2.7.tgz
  !wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
  !tar xf spark-3.0.0-bin-hadoop2.7.tgz
  pass

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,391 

In [ ]:
# Set up Spark
if first_time:
  !pip install -q findspark
  !pip install py4j
  pass

!export JAVA_HOME=$(/usr/lib/jvm/java-8-openjdk-amd64 -v 1.8)
! echo $JAVA_HOME
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"

import findspark
# findspark.init("spark-2.4.7-bin-hadoop2.7")# SPARK_HOME
findspark.init("spark-3.0.0-bin-hadoop2.7")# SPARK_HOME
if first_time:
  !pip install --upgrade PyArrow
  pass
import os
os.environ["PYSPARK_PYTHON"] = "python3"
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"


from pyspark.sql import SparkSession


     |████████████████████████████████| 204kB 5.2MB/s 
/bin/bash: /usr/lib/jvm/java-8-openjdk-amd64: Is a directory

     |████████████████████████████████| 20.7MB 1.3MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [ ]:
  # spark = SparkSession \
  #     .builder \
  #     .appName("AvazuCTR") \
  #     .config("spark.some.config.option", "some-value") \
  #     .getOrCreate()
# --------customized full, when using petastorm
spark = SparkSession \
        .builder \
        .master("local[*]")\
        .appName("AvazuCTR") \
        .config("spark.driver.memory","24g") \
        .config('spark.kryoserializer.buffer.max.mb',2047) \
        .config('spark.memory.offHeap.size',"16g") \
        .getOrCreate()


    # .config("spark.driver.memory","6g")  \
    # .config("spark.memory.fration","0.9") \
#reference: https://stackoverflow.com/questions/21138751/spark-java-lang-outofmemoryerror-java-heap-space

## Read data

In [ ]:
# copy parquet files from Google Drive
  # full data
if copy_full_data:
  !cp -R /content/drive/MyDrive/LAI_OFFER/BIG_DATA/AvazuCTR/Data/parquet/train.csv .
  pass
if copy_sample_data:
# sample data
  !cp -R /content/drive/MyDrive/LAI_OFFER/BIG_DATA/AvazuCTR/Data/parquet/filtered_train.csv .
  pass
  
# a larger number of partitions makes a partition smaller than the allocated excutor memory ensuring performance
if full_data:
# full data:
  impression = spark.read.parquet("train.csv")#.repartition(4) #repartition(64)
else:
# sample data:
  impression = spark.read.parquet("filtered_train.csv")#.repartition(5) #repartition(64) # works for small data





===========NEED TO RUN ALL ABOVE BEFORE RUNNING IN TERMINAL==============

### get column 'hr'

In [ ]:
impression = impression.selectExpr("*", "substr(hour, 7) as hr")
# impression.show(5)

### get column 'date' (int)
 used for splitting train and test data

In [ ]:
# get a new column 'date' (string)
impression = impression.selectExpr("*", "substr(hour, 3, 4) as date")
# final_df.show(5)
# final_df.registerTempTable('final_df')
# get column of date of int type
impression = impression.withColumn('date', impression.date.cast('int'))
impression.createOrReplaceTempView("impression")
# impression.show(5)

### get column 'dayOfWeek'


In [ ]:
impression.show(5)

+--------+--------------------+-----+--------+----+----------+--------+-----------+-------------+--------+----------+------------+---------+---------+------------+-----------+----------------+-----+---+---+----+---+---+------+---+---+----+
|     _c0|                  id|click|    hour|  C1|banner_pos| site_id|site_domain|site_category|  app_id|app_domain|app_category|device_id|device_ip|device_model|device_type|device_conn_type|  C14|C15|C16| C17|C18|C19|   C20|C21| hr|date|
+--------+--------------------+-----+--------+----+----------+--------+-----------+-------------+--------+----------+------------+---------+---------+------------+-----------+----------------+-----+---+---+----+---+---+------+---+---+----+
|21193144|1.965071826656588...|    0|14102608|1005|         0|6399eda6|   968765cd|     f028772b|ecad2386|  7801e8d9|    07d7df22| a99f214a| 191f29f8|    b4c772f2|          1|               0|21153|320| 50|2420|  2| 35|    -1| 69| 08|1026|
|21946717|1.007752156511895...|    1|141

In [ ]:
from pyspark.sql.functions import to_date, to_timestamp, hour, dayofweek
from pyspark.sql.types import StringType
from pyspark.sql.functions import col as column
impression = impression.withColumn('timestamp',to_timestamp(impression.hour.cast('string'),format = 'yyMMddHH'))
impression = impression.withColumn('dayOfWeek',dayofweek(column('timestamp')))
impression.show(5)

+--------+--------------------+-----+--------+----+----------+--------+-----------+-------------+--------+----------+------------+---------+---------+------------+-----------+----------------+-----+---+---+----+---+---+------+---+---+----+-------------------+---------+
|     _c0|                  id|click|    hour|  C1|banner_pos| site_id|site_domain|site_category|  app_id|app_domain|app_category|device_id|device_ip|device_model|device_type|device_conn_type|  C14|C15|C16| C17|C18|C19|   C20|C21| hr|date|          timestamp|dayOfWeek|
+--------+--------------------+-----+--------+----+----------+--------+-----------+-------------+--------+----------+------------+---------+---------+------------+-----------+----------------+-----+---+---+----+---+---+------+---+---+----+-------------------+---------+
|21193144|1.965071826656588...|    0|14102608|1005|         0|6399eda6|   968765cd|     f028772b|ecad2386|  7801e8d9|    07d7df22| a99f214a| 191f29f8|    b4c772f2|          1|               

In [ ]:
impression.rdd.getNumPartitions()

4

In [ ]:
impression.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- id: double (nullable = true)
 |-- click: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- C1: integer (nullable = true)
 |-- banner_pos: integer (nullable = true)
 |-- site_id: string (nullable = true)
 |-- site_domain: string (nullable = true)
 |-- site_category: string (nullable = true)
 |-- app_id: string (nullable = true)
 |-- app_domain: string (nullable = true)
 |-- app_category: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- device_ip: string (nullable = true)
 |-- device_model: string (nullable = true)
 |-- device_type: integer (nullable = true)
 |-- device_conn_type: integer (nullable = true)
 |-- C14: integer (nullable = true)
 |-- C15: integer (nullable = true)
 |-- C16: integer (nullable = true)
 |-- C17: integer (nullable = true)
 |-- C18: integer (nullable = true)
 |-- C19: integer (nullable = true)
 |-- C20: integer (nullable = true)
 |-- C21: integer (nullable = true)
 |-- hr: strin

In [ ]:
impression.count()

404290

In [ ]:
# from pyspark.sql.types import LongType, DecimalType,StringType
# impression = impression.withColumn('id',impression.id.cast(StringType()))
# impression.printSchema()
# impression.show(5)

## SPECIFY WIDE and DEEP features

### WIDE features

In [ ]:
wide_cols = impression.columns
if full_data:
  to_remove = ['click'] + ['hour','id','date','timestamp']
else:
  to_remove = ['click'] + ['hour','id','date','timestamp','_c0']
for col in to_remove:
  wide_cols.remove(col)
wide_cols

['C1',
 'banner_pos',
 'site_id',
 'site_domain',
 'site_category',
 'app_id',
 'app_domain',
 'app_category',
 'device_id',
 'device_ip',
 'device_model',
 'device_type',
 'device_conn_type',
 'C14',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C20',
 'C21',
 'hr',
 'dayOfWeek']

#### Interactive features

In [ ]:
# interactive_features= [('device_id','device_conn_type'),('device_id','site_id')]
interactive_features = []

### DEEP features
Deep part in wide and deep learning is used for generalizaiton. Therefore, the extreamly high sparse data are not useful??(Need to verify my understanding...)


In [ ]:
import pickle
embeddings_cols = impression.columns
# to_remove_emb = ['device_id','device_ip','device_model','site_id','site_domain','app_id','C14']
to_remove_emb = ['device_id','device_ip']
if full_data:
           #orig label + added_augmenting + original unuseful 
  always = ['click'] + ['date','timestamp'] + ['id','hour'] 
else:      #'click' + added_augmenting + original unuseful 
  always = ['click'] + ['date','timestamp'] + ['id','hour','_c0']
to_remove_emb = to_remove_emb + always
for col in to_remove_emb:
  embeddings_cols.remove(col)


print(len(embeddings_cols))
embeddings_cols

21


['C1',
 'banner_pos',
 'site_id',
 'site_domain',
 'site_category',
 'app_id',
 'app_domain',
 'app_category',
 'device_model',
 'device_type',
 'device_conn_type',
 'C14',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C20',
 'C21',
 'hr',
 'dayOfWeek']

### features positions

In [ ]:
# get intersection of wide features and deep features.

# wide_deep_intersection =[col for col in wide_cols if col in embeddings_cols]
# wide_only = [col for col in wide_cols if col not in embeddings_cols]
# deep_only = [col for col in embeddings_cols if col not in wide_cols]
# wide_deep_union = wide_only + wide_deep_intersection + deep_only

In [ ]:
num_wide_features = len(wide_cols)
num_wide_features

23

## FEATURE ENGINEERING

Features used for deep part are engineered here. Mainly string indexing the feature values.

In [ ]:
import time
import numpy as np
import pandas as pd
from IPython.display import display # for displaying multiple  dataframes from a single cell
import seaborn as sns
import matplotlib.pyplot as plt
import math
import sys
%matplotlib inline

# convert string coluumns to integer type before performing OHE

from pyspark.ml.feature import OneHotEncoder, StringIndexer

### Feature hashing
Use UDF to performe feature hashing, output is the feature index

In [ ]:
# original features:
from pyspark.sql.functions import col as column
feature_hashed = impression
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
D = 2**22
for col_name in wide_cols:
  def feature_hashing(feature_val):
    return abs(hash( col_name +'_'+ str(feature_val))) % D #use '_' in between col name and feature val, 0221
  udf_feature_hash = udf(feature_hashing,IntegerType())
  feature_hashed = feature_hashed.withColumn(col_name+'_hash',udf_feature_hash(column(col_name)) )
# feature_hashed.show(10)

In [ ]:
feature_hashed.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- id: double (nullable = true)
 |-- click: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- C1: integer (nullable = true)
 |-- banner_pos: integer (nullable = true)
 |-- site_id: string (nullable = true)
 |-- site_domain: string (nullable = true)
 |-- site_category: string (nullable = true)
 |-- app_id: string (nullable = true)
 |-- app_domain: string (nullable = true)
 |-- app_category: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- device_ip: string (nullable = true)
 |-- device_model: string (nullable = true)
 |-- device_type: integer (nullable = true)
 |-- device_conn_type: integer (nullable = true)
 |-- C14: integer (nullable = true)
 |-- C15: integer (nullable = true)
 |-- C16: integer (nullable = true)
 |-- C17: integer (nullable = true)
 |-- C18: integer (nullable = true)
 |-- C19: integer (nullable = true)
 |-- C20: integer (nullable = true)
 |-- C21: integer (nullable = true)
 |-- hr: strin

### Feature string indexing
For deep embedding features

In [ ]:
inputCols_strI = embeddings_cols
outputCols_strI = [col+'_strI' for col in inputCols_strI]

stringIndexer_1_estimator = StringIndexer(inputCols = inputCols_strI, outputCols = outputCols_strI)
stringIndexer_1_transformer = stringIndexer_1_estimator.fit(feature_hashed)
feature_indexd = stringIndexer_1_transformer.transform(feature_hashed)
# feature_indexd.show(5)

In [ ]:
# processed_output = [col+'_hash' for col in wide_cols] + outputCols_strI
to_remove = wide_cols + embeddings_cols
to_remove = list(set(to_remove))
to_remove

['site_category',
 'app_id',
 'site_id',
 'app_domain',
 'C16',
 'C15',
 'app_category',
 'C17',
 'hr',
 'device_model',
 'C1',
 'C20',
 'C19',
 'device_id',
 'device_ip',
 'dayOfWeek',
 'device_type',
 'C14',
 'C18',
 'site_domain',
 'banner_pos',
 'device_conn_type',
 'C21']

In [ ]:
all_cols = feature_indexd.columns
kept_cols = [col for col in all_cols if col not in to_remove]
feature_indexd = feature_indexd.select(kept_cols)
feature_indexd.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- id: double (nullable = true)
 |-- click: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- date: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- C1_hash: integer (nullable = true)
 |-- banner_pos_hash: integer (nullable = true)
 |-- site_id_hash: integer (nullable = true)
 |-- site_domain_hash: integer (nullable = true)
 |-- site_category_hash: integer (nullable = true)
 |-- app_id_hash: integer (nullable = true)
 |-- app_domain_hash: integer (nullable = true)
 |-- app_category_hash: integer (nullable = true)
 |-- device_id_hash: integer (nullable = true)
 |-- device_ip_hash: integer (nullable = true)
 |-- device_model_hash: integer (nullable = true)
 |-- device_type_hash: integer (nullable = true)
 |-- device_conn_type_hash: integer (nullable = true)
 |-- C14_hash: integer (nullable = true)
 |-- C15_hash: integer (nullable = true)
 |-- C16_hash: integer (nullable = true)
 |-- C17_hash: integer (

In [ ]:
def dtype_col(col):
  for col_i,dtype in feature_indexd.dtypes:
    if col == col_i:
      return dtype
dtype_col('device_conn_type_hash')

'int'

### dtype conversion

In [ ]:
# to save space, convert double to int, but later vectorAssembler will put int back to float...
from pyspark.sql.types import FloatType,IntegerType
temp_cols = feature_indexd.columns
temp_cols.remove('id') # keep 'id' being double
for col in temp_cols:
  if dtype_col(col) == 'double':
    feature_indexd = feature_indexd.withColumn(col,  column(col).cast(IntegerType()))
feature_indexd.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- click: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- date: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- C1_hash: integer (nullable = true)
 |-- banner_pos_hash: integer (nullable = true)
 |-- site_id_hash: integer (nullable = true)
 |-- site_domain_hash: integer (nullable = true)
 |-- site_category_hash: integer (nullable = true)
 |-- app_id_hash: integer (nullable = true)
 |-- app_domain_hash: integer (nullable = true)
 |-- app_category_hash: integer (nullable = true)
 |-- device_id_hash: integer (nullable = true)
 |-- device_ip_hash: integer (nullable = true)
 |-- device_model_hash: integer (nullable = true)
 |-- device_type_hash: integer (nullable = true)
 |-- device_conn_type_hash: integer (nullable = true)
 |-- C14_hash: integer (nullable = true)
 |-- C15_hash: integer (nullable = true)
 |-- C16_hash: integer (nullable = true)
 |-- C17_hash: integer 

### label col: int -> float

In [ ]:
# The [click] column is our label 

labelStringIndexer_estimator = StringIndexer(inputCol = "click", outputCol = "label")

labelStringIndexer_transformer = labelStringIndexer_estimator.fit(feature_indexd)
label_indexed = labelStringIndexer_transformer.transform(feature_indexd)

In [ ]:
# change double to float to save space
label_indexed = label_indexed.withColumn('label',label_indexed.label.cast('float'))
label_indexed.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- click: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- date: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- C1_hash: integer (nullable = true)
 |-- banner_pos_hash: integer (nullable = true)
 |-- site_id_hash: integer (nullable = true)
 |-- site_domain_hash: integer (nullable = true)
 |-- site_category_hash: integer (nullable = true)
 |-- app_id_hash: integer (nullable = true)
 |-- app_domain_hash: integer (nullable = true)
 |-- app_category_hash: integer (nullable = true)
 |-- device_id_hash: integer (nullable = true)
 |-- device_ip_hash: integer (nullable = true)
 |-- device_model_hash: integer (nullable = true)
 |-- device_type_hash: integer (nullable = true)
 |-- device_conn_type_hash: integer (nullable = true)
 |-- C14_hash: integer (nullable = true)
 |-- C15_hash: integer (nullable = true)
 |-- C16_hash: integer (nullable = true)
 |-- C17_hash: integer 

### Vector Assembling

Assembling hashed wide features, embedding features and labels

#### hashed wide

In [ ]:
#VectorAssembler convert all numercial type to float. So int above will be converted to float
from pyspark.ml.feature import VectorAssembler
inputCols_assember = [col + '_hash'  for col in wide_cols] 
outputCol_assember = "hashed_wide"
vectorAssembler = VectorAssembler(inputCols = inputCols_assember, outputCol = outputCol_assember)

hashed_wide_assembled = vectorAssembler.transform(label_indexed)
# hashed_wide_assembled.show(5)

#### embedding features

In [ ]:
from pyspark.ml.feature import VectorAssembler
inputCols_assember = outputCols_strI 
outputCol_assember = "embedding_indexed"
vectorAssembler = VectorAssembler(inputCols = inputCols_assember, outputCol = outputCol_assember)

embedding_assembled = vectorAssembler.transform(hashed_wide_assembled)
# embedding_assembled.show(5)

### Throw away unuseful cols

In [ ]:
# ------keep only useful columns, not label are with 'emb_features'
loader_cols = ['id','date'] + ['hashed_wide'] + ['embedding_indexed'] + ['label'] # 'hour'
df_useful_cols = embedding_assembled.select(loader_cols)

In [ ]:
df_useful_cols.select('date').distinct().show()

+----+
|date|
+----+
|1025|
|1030|
|1021|
|1026|
|1028|
|1029|
|1022|
|1027|
|1023|
|1024|
+----+



### train-test splitting and repartition by date

Repartition here was to sort data by data, but the partitionBy below turned out not helpful, because petastorm will mess up the ordering when conveting data to dataloder...

In [ ]:
# seprate train and test data using 'date'
train_df = df_useful_cols.filter(df_useful_cols.date<=1029)
test_df = df_useful_cols.filter(df_useful_cols.date>1029)

SOME READ ABOUT ORDER PRESERVATION
https://towardsdatascience.com/notes-about-saving-data-with-spark-3-0-86ba85ca2b71

In [ ]:
train_df = train_df.repartitionByRange(9,'date')
test_df = test_df.repartitionByRange(1,'date')
# train_df = train_df.orderBy('date',ascending = True)
# test_df = test_df.orderBy('date',ascending = True)

In [ ]:
# train_df.show()

### select columns if necessary

In [ ]:
# loader_cols.remove('hour')
train_df = train_df.select(loader_cols)
test_df = test_df.select(loader_cols)

In [ ]:
test_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- date: integer (nullable = true)
 |-- hashed_wide: vector (nullable = true)
 |-- embedding_indexed: vector (nullable = true)
 |-- label: float (nullable = false)



## SAVE DATA 

In [ ]:
import pickle
path_processed_features = '/content/drive/MyDrive/LAI_OFFER/BIG_DATA/AvazuCTR/WDL_FTRL/data_wDash/1_SparkProcess_AvazuCTR.pickle'
with open(path_processed_features,'wb') as handle:  
  pickle.dump(wide_cols, handle,protocol=pickle.HIGHEST_PROTOCOL)
  pickle.dump(interactive_features, handle,protocol=pickle.HIGHEST_PROTOCOL)
  pickle.dump(D, handle,protocol=pickle.HIGHEST_PROTOCOL)

  pickle.dump(embeddings_cols, handle,protocol=pickle.HIGHEST_PROTOCOL)
  pickle.dump(loader_cols, handle,protocol=pickle.HIGHEST_PROTOCOL)
  
print('len wide_cols',len(wide_cols))
print('len interactive_features',len(interactive_features))
print('D',D)
print('len embeddings_cols',len(embeddings_cols))
print('len(loader_cols)',len(loader_cols))

print('-----------')

print('- wide_cols',wide_cols)
print('- interactive_features',interactive_features)
print('- D',D)
print('- embeddings_cols',embeddings_cols)
print('- loader_cols',loader_cols)

len wide_cols 23
len interactive_features 0
D 4194304
len embeddings_cols 21
len(loader_cols) 5
-----------
- wide_cols ['C1', 'banner_pos', 'site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category', 'device_id', 'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'hr', 'dayOfWeek']
- interactive_features []
- D 4194304
- embeddings_cols ['C1', 'banner_pos', 'site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category', 'device_model', 'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'hr', 'dayOfWeek']
- loader_cols ['id', 'date', 'hashed_wide', 'embedding_indexed', 'label']


In [ ]:
train_df.rdd.getNumPartitions()

9

In [ ]:
# write data to local machine, the partitionBy below turned out not helpful, because petastorm will mess up
# the ordering when conveting data to dataloder...
if full_data:
  train_df.write.mode('overwrite').partitionBy('date').parquet("train_df_parquet")
  test_df.write.mode('overwrite').partitionBy('date').parquet("test_df_parquet")
else:
  train_df.write.mode('overwrite').partitionBy('date').parquet("train_df_parquet_filtered")
  test_df.write.mode('overwrite').partitionBy('date').parquet("test_df_parquet_filtered")

==========NEED TO COMMMENT OUT SCRIPT BELOW BEFORE RUNNING IN TERMINAL===============





In [ ]:
# copy data from local machine to google drive
if full_data:
  !cp -r train_df_parquet /content/drive/MyDrive/LAI_OFFER/BIG_DATA/AvazuCTR/WDL_FTRL/data_wDash/train_df_parquet
  !cp -r test_df_parquet  /content/drive/MyDrive/LAI_OFFER/BIG_DATA/AvazuCTR/WDL_FTRL/data_wDash/test_df_parquet
  pass
else:
  !cp -r train_df_parquet_filtered /content/drive/MyDrive/LAI_OFFER/BIG_DATA/AvazuCTR/WDL_FTRL/data_wDash/train_df_parquet_filtered
  !cp -r test_df_parquet_filtered  /content/drive/MyDrive/LAI_OFFER/BIG_DATA/AvazuCTR/WDL_FTRL/data_wDash/test_df_parquet_filtered
  pass